In [1]:
import os
import warnings
import pandas as pd

In [2]:
def loadRes(problem, sizes, methods, new=False, limit60=False):
    # init a list to store processed results
    results = []
    # loop through each size
    for s in sizes:
        for m in methods:
            # retrive metrics
            metrics = retriveData(problem, s, m, new, limit60)
            # append the results to the list
            results.append(metrics)
    # convert into a Pandas DataFrame
    result_df = pd.DataFrame(results)
    # reshape
    result_df = reshapeDataframe(result_df, problem, sizes)
    return result_df

def retriveData(problem, size, method, new, limit60):
    # method map
    method_mapping = {
    "cls": "RC", "thd": "LT", "exact": "EX", 
    "rel": "RR", "root": "N1", "lrn": "RL", "ste": "RS",
    "cls50": "RC", "thd50": "LT", "lrn50": "RL", "ste50": "RS",
    "cls100": "RC", "thd100": "LT", "lrn100": "RL", "ste100": "RS",
    "cls200": "RC", "thd200": "LT", "lrn200": "RL", "ste200": "RS"
    }
    # dir and file
    if limit60 and method in ["exact", "rel", "root"]:
        data_path = "./result60/"
    else:
        data_path = "./result/"
    if problem == "rb":
        if new:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}_new.csv")
        else:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}.csv")
    else:
        if new:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}-{size}_new.csv")
        else:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}-{size}.csv")
    #print(csv_file)
    if not os.path.exists(csv_file):
        # output a warning
        warnings.warn(f"File not found: {csv_file}")
        # return None values for all metrics
        return {
            "Method": method_mapping[method],
            "Problem Size": size,
            "Obj Mean": None,
            "Obj Median": None,
            "% Infeasible": None,
            "% Unsolved": None,
            "Time (Sec)": None,
            "Viol Mean": None,
            "Viol Max": None,
            "Num Viol Mean": None
        }
    # load data
    df = pd.read_csv(csv_file)
    # calculate the required metrics
    obj_mean = df["Obj Val"].mean()  # mean of the objective values
    obj_median = df["Obj Val"].median()  # median of the objective values
    if "Num Violations" in df.columns:
        percent_infeasible = (df["Num Violations"] > 0).mean() * 100  # percentage of infeasible instances
    else:
        percent_infeasible = (df["Constraints Viol"] > 0).mean() * 100  # percentage of infeasible instances
    #mean_viols = df["Mean Violation"].mean()  # average mean violation
    #max_viols = df["Max Violation"].max()  # maximum violation
    #num_viols = df["Num Violations"].mean()  # average number of violations
    percent_unsolved = df["Obj Val"].isna().mean() * 100  # percentage of unsolved cases
    time_mean = df["Elapsed Time"].mean()  # Mean of the elapsed solving times
    # append the results to the list
    metrics = {
        "Method": method_mapping[method],
        "Problem Size": size,
        "Obj Mean": obj_mean,
        "Obj Median": obj_median,
        "% Infeasible": percent_infeasible,
        "% Unsolved": percent_unsolved,
        "Time (Sec)": time_mean,
        #"Viol Mean": mean_viols,
        #"Viol Max": max_viols,
        #"Num Viol Mean": num_viols
    }
    return metrics

def reshapeDataframe(df, problem, sizes):
    # define the metrics to include
    metrics = ["Obj Mean", "Obj Median", "% Infeasible", "% Unsolved", "Time (Sec)"]#, "Viol Mean", "Viol Max", "Num Viol Mean"]
    # define the method to include
    methods = ["RC", "LT", "EX", "RR", "N1", "RL", "RS"]
    # reshape
    reshaped_data = []
    for method in methods:
        for metric in metrics:
            row = {"Method": method, "Metric": metric}
            for s in sizes:
                value = df[(df["Method"] == method) & (df["Problem Size"] == s)][metric].values
                # check if the value has exactly one element
                if len(value) != 1:
                    raise ValueError(f"Unexpected number of values ({len(value)}) for Method: {method}, Metric: {metric}, Problem Size: {s}")
                if problem == "rb":
                    row[f"{s*2}×4"] = value[0]
                else:
                    row[f"{s}×{s}"] = value[0]
            reshaped_data.append(row)
    # convert to a DataFrame
    reshaped_df = pd.DataFrame(reshaped_data)
    return reshaped_df

### Convex Quadratic

In [3]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls200", "thd200", "exact", "rel", "root", "lrn200", "ste200"]
loadRes(problem="cq", sizes=sizes, methods=methods)

,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.802874,-1.475846e+00,-4.151028e+00,-1.242196e+01,-1.356417e+01,-30.799130,-71.372707,-120.377681
1,RC,Obj Median,0.647734,-1.718971e+00,-4.165983e+00,-1.241575e+01,-1.363406e+01,-30.788936,-71.476421,-120.491720
2,RC,% Infeasible,0.000000,3.000000e+00,4.000000e+00,0.000000e+00,4.000000e+00,1.000000,4.000000,3.000000
3,RC,% Unsolved,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001859,1.884835e-03,1.880178e-03,1.913416e-03,2.108037e-03,0.002502,0.002571,0.004519
5,LT,Obj Mean,0.743269,-1.615401e+00,-4.164674e+00,-1.235466e+01,-1.339545e+01,-29.728064,-71.350230,-114.433595
6,LT,Obj Median,0.553628,-1.928284e+00,-4.202993e+00,-1.235223e+01,-1.352658e+01,-29.808278,-71.600544,-114.521791
7,LT,% Infeasible,1.000000,1.000000e+00,0.000000e+00,4.000000e+00,3.000000e+00,0.000000,3.000000,1.000000
8,LT,% Unsolved,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
9,LT,Time (Sec),0.001876,1.898167e-03,1.938312e-03,1.940231e-03,2.285924e-03,0.002169,0.002570,0.004580


### Simple Non-Convex (Perturbed A)

In [4]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls100", "thd100", "exact", "rel", "root", "lrn100", "ste100"]
loadRes(problem="nc", sizes=sizes, methods=methods, new=True, limit60=False)

C:\Users\lucas\AppData\Local\Temp\ipykernel_5192\3386157782.py:44: UserWarning: File not found: ./result/nc_exact_20-20_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_5192\3386157782.py:44: UserWarning: File not found: ./result/nc_exact_50-50_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_5192\3386157782.py:44: UserWarning: File not found: ./result/nc_root_500-500_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_5192\3386157782.py:44: UserWarning: File not found: ./result/nc_exact_1000-1000_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_5192\3386157782.py:44: UserWarning: File not found: ./result/nc_root_1000-1000_new.csv
  warnings.warn(f"File not found: {csv_file}")


,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.382745,1.101563,0.227695,7.712910e-01,1.663565,1.471528,0.526173,1.422244
1,RC,Obj Median,0.239879,0.837723,0.216590,7.519165e-01,1.594214,1.436150,0.526173,0.809124
2,RC,% Infeasible,1.000000,3.000000,0.000000,2.000000e+00,0.000000,1.000000,4.000000,3.000000
3,RC,% Unsolved,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001919,0.001936,0.001928,2.014108e-03,0.002193,0.002170,0.002871,0.003962
5,LT,Obj Mean,0.359111,0.909994,0.194875,5.796574e-01,0.668875,-0.356054,-1.373667,-3.743579
6,LT,Obj Median,0.225594,0.682508,0.175219,5.663435e-01,0.648688,-0.373353,-1.593921,-3.716251
7,LT,% Infeasible,0.000000,2.000000,1.000000,2.000000e+00,4.000000,0.000000,2.000000,1.000000
8,LT,% Unsolved,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
9,LT,Time (Sec),0.001949,0.001943,0.001912,1.978652e-03,0.002149,0.002340,0.002877,0.004997


### Rosenbrock

In [5]:
sizes = [1, 10, 100, 1000, 10000]
methods = ["cls100", "thd100", "exact", "rel", "root", "lrn100", "ste100"]
loadRes(problem="rb", sizes=sizes, methods=methods, limit60=False)

C:\Users\lucas\AppData\Local\Temp\ipykernel_5192\3386157782.py:44: UserWarning: File not found: ./result/rb_exact_1000.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_5192\3386157782.py:44: UserWarning: File not found: ./result/rb_exact_10000.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_5192\3386157782.py:44: UserWarning: File not found: ./result/rb_rel_10000.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_5192\3386157782.py:44: UserWarning: File not found: ./result/rb_root_10000.csv
  warnings.warn(f"File not found: {csv_file}")


,Method,Metric,2×4,20×4,200×4,2000×4,20000×4
0,RC,Obj Mean,23.265538,59.393365,5.035068e+02,5.938372e+03,66883.203419
1,RC,Obj Median,21.475520,48.857842,4.617061e+02,5.792519e+03,66797.105791
2,RC,% Infeasible,3.000000,0.000000,1.000000e+00,1.000000e+00,24.000000
3,RC,% Unsolved,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
4,RC,Time (Sec),0.001926,0.001944,2.087076e-03,3.257055e-03,0.012133
5,LT,Obj Mean,23.182455,62.508514,6.227751e+02,5.611775e+03,47622.180004
6,LT,Obj Median,20.796687,63.404079,6.260387e+02,5.557821e+03,34518.870360
7,LT,% Infeasible,2.000000,0.000000,0.000000e+00,3.000000e+00,34.000000
8,LT,% Unsolved,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
9,LT,Time (Sec),0.001930,0.001988,2.615485e-03,2.994518e-03,0.012724
